Locked in. Next lab is from:  
### 📁 `02_computer_vision`  
Let’s go deep and visual 👁️

---

# 🧠 `07_lab_cnn_feature_maps_visualization.ipynb`  
### 🧪 Purpose:  
> **Visualize how CNNs "see"** — layer-by-layer.  
Inspect learned filters and activation maps to gain intuition on what convolutional layers extract at different depths.

---

## 💻 Target Setup

| Requirement        | Designed For     |
|--------------------|------------------|
| Device             | CPU / Colab GPU (T4) ✅  
| RAM                | < 2GB ✅  
| Libs               | PyTorch, matplotlib ✅  
| Dataset            | CIFAR-10 (built-in) ✅  

---

## 🧪 Step-by-Step Breakdown

---

### ✅ Section 1: Imports & Setup

```python
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
```

---

### 🖼️ Section 2: Load CIFAR-10 Sample

```python
# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

classes = testset.classes
```

---

### 🧠 Section 3: Load Pretrained CNN (e.g., ResNet18)

```python
from torchvision.models import resnet18

model = resnet18(pretrained=True)
model.eval()
```

---

### 🔬 Section 4: Hook to Extract Feature Maps

```python
feature_maps = {}

def get_activation(name):
    def hook(model, input, output):
        feature_maps[name] = output.detach()
    return hook

# Register hook on early conv layer
model.layer1[0].conv1.register_forward_hook(get_activation("layer1.0.conv1"))
model.layer2[0].conv1.register_forward_hook(get_activation("layer2.0.conv1"))
```

---

### 🖼️ Section 5: Visualize Input Image

```python
# Load sample
dataiter = iter(testloader)
images, labels = next(dataiter)

# Unnormalize for display
img = images[0].permute(1, 2, 0) * 0.5 + 0.5
plt.imshow(img)
plt.title(f"Input Image: {classes[labels[0]]}")
plt.axis("off")
plt.show()
```

---

### 📡 Section 6: Forward Pass & Capture

```python
with torch.no_grad():
    _ = model(images)
```

---

### 🎨 Section 7: Visualize Feature Maps

```python
def show_feature_maps(layer_name, num_channels=6):
    fmap = feature_maps[layer_name][0]  # [C, H, W]
    fig, axes = plt.subplots(1, num_channels, figsize=(15, 5))
    for i in range(num_channels):
        axes[i].imshow(fmap[i].cpu(), cmap='viridis')
        axes[i].axis("off")
        axes[i].set_title(f"Filter {i}")
    plt.suptitle(f"Feature Maps - {layer_name}")
    plt.show()

# Try:
show_feature_maps("layer1.0.conv1", num_channels=6)
show_feature_maps("layer2.0.conv1", num_channels=6)
```

---

## 🔍 Bonus: Visualize Filters Themselves (Weights)

```python
# Extract filters from conv1
filters = model.conv1.weight.data.clone()

fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for i in range(16):
    f = filters[i]
    f_min, f_max = f.min(), f.max()
    f_img = (f - f_min) / (f_max - f_min)
    axes[i // 4, i % 4].imshow(f_img.permute(1, 2, 0).cpu())
    axes[i // 4, i % 4].axis("off")

plt.suptitle("First Conv Layer Filters")
plt.show()
```

---

## ✅ Wrap-Up Checklist

| Concept                  | Covered |
|--------------------------|---------|
| Hooking into layers      | ✅       |
| Forward-pass inspection  | ✅       |
| Visualizing activations  | ✅       |
| Understanding filters    | ✅       |
| No GPU dependency        | ✅       |
| Colab compatible         | ✅       |

---

## 🧠 What You Learned

- CNNs extract **low-level patterns** in early layers (edges, colors)  
- Deeper layers encode **shapes, textures, semantics**  
- Hooks allow **peeking inside the model brain**

---

Ready for `08_lab_data_augmentation_comparison.ipynb` next?  
We'll test flips, cutout, mixup — and actually **track how accuracy shifts**.